In [11]:
import numpy as np 
import networkx as nx
import collections
import openai
import json
import os
import pprint
import matplotlib.pyplot as plt
import random
import copy
import pandas as pd

with open('params.json') as f:
    params = json.load(f)

openai.api_key = params['OPENAI_API_KEY']
openai.organization = params['OPENAI_ORG']

def get_response(prompt, model='gpt-4'):
    result = openai.ChatCompletion.create(
    model=model,
    messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
    ])

    return result.choices[0]['message']['content']

In [12]:
def generate_karate_club_agents(num_agents):

    agents_prompt = f"""
        # Task
        Your task is to generate the opinions of {num_agents} students.
        The opinions are generated as follows:

        Each student attends the karate club. The karate club instructor, John Doe, wants to raise prices, claiming the authority to set his own lesson fees, since he is the instructor.
        The karate club administrator, Richard Roe, wants to keep prices low, arguing that the club should be affordable to all, claiming his authority as the club's administrator.

        # Output
        You should output a list of JSON objects, formatted as follows

        [
            {{
                "name" : "Student <id>" (e.g. "Student 1"),
                "private_opinion": the student's private opnion regarding the lesson fees and a persuasive rationale for the opinion.
                "value" : (float) a numeric value ranging from 0 to 100, indicating the student's private opinion sentiment regarding the lesson fees (0 = support low fees, 100 = support high fees)
            }}, ...
        ]

        
    ```json
    """

    agents = json.loads(get_response(agents_prompt).split('```')[0])

    for agent in agents:
        agent['public_opinion'] = agent['private_opinion']
        agent['is_student'] = True

    john_doe = {
        "name" : "John Doe",
        "private_opinion" : "I think the karate club should raise prices, because I am the instructor and I should be able to set my own lesson fees.",
        "public_opinion" : "I think the karate club should raise prices, because I am the instructor and I should be able to set my own lesson fees.",
        "is_student" : False,
        "value" : 100
    }
        
    richard_roe = {
        "name" : "Richard Roe",
        "private_opinion" : "I think the karate club should keep prices low, because I am the administrator and the club should be affordable to all.",
        "public_opinion" : "I think the karate club should keep prices low, because I am the administrator and the club should be affordable to all.",
        "is_student" : False,
        "value" : 0
    }

    agents.append(john_doe)
    agents.append(richard_roe)

    with open('outputs_simple/agents.json', 'w+') as f:
        f.write(json.dumps(agents, indent=4))

    return agents

def generate_karate_club_locations(num_classes=3):

    locations_prompt = f"""
    # Task 
    You are an AI assistant and your task is to generate a short description of some locations regarding a university. 
    The locations are the following: 
    * {num_classes} academic classes at the university. The classes can be from a diverse set of subjects, such as math, physics, chemistry, biology, computer science, psychology, law, journalism, policital scienceetc.
    * A karate studio belonging to John Doe.
    * A bar located accross the street from the university campus.
    * A university rathskeller.
    * A dojo for holding open karate tournaments through the area at private karate studios.
    * A dojo for intercollegiate karate tournaments belonging to the university.


    # Output
    You must format your output as a list of JSON objects with the following fields:

    [
        {{
            "name" : name of location,
            "description" : description of location,
            "type" : type of location (e.g. academic class, karate studio, bar, etc.)
        }},
        ...
    ]

    ## Notes
    * You must output JSON objects for each location.
    
    ```json
    """

    response = get_response(locations_prompt).split('```')[0]

    with open('outputs_simple/locations.json', 'w+') as f:
        f.write(response)

    return json.loads(response)

def create_world(num_classes=3, num_agents=3):

    if os.path.exists('outputs_simple/locations.json'):
        with open('outputs_simple/locations.json') as f:
            locations = json.load(f)
    else:
        locations = generate_karate_club_locations(num_classes=num_classes)

    locations = dict([(location['name'], location) for location in locations])

    if os.path.exists('outputs_simple/agents.json'):
        with open('outputs_simple/agents.json') as f:
            agents = json.load(f)
    else:
        agents = generate_karate_club_agents(num_agents=num_agents)

    agents = dict([(agent['name'], agent) for agent in agents])

    config = {
        "world_name" : "University of Anytown",
        "locations" : locations,
        "agents" : agents
    }

    with open('outputs_simple/config.json', 'w+') as f:
        f.write(json.dumps(config, indent=4))


    return config

def plot_network(G):
    fig, ax = plt.subplots(figsize=(10, 10))
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    pos = nx.spectral_layout(G)
    nx.draw(G, pos, node_size=10, edgelist=edges, edge_color=weights, width=5.0, edge_cmap=plt.cm.Blues, ax=ax, arrows=True)

In [13]:
# Possible environments
# 1. Pick someone at random and talk to them
# 2. Have access to everyone in the same location, and consolidate your opinion based on the opinions of everyone in the same location
# 3. when they make a decision whom to talk to, you ask the agent whom to talk to

def network_formation(locations, agents, num_iters, p_change=0.2, filename='outputs_simple/snapshots.jsonl'):

    if os.path.exists(filename):
        print(f'Loading snapshot from {filename}')
        f = open(filename, 'a+')

        lines = f.readlines()

        snapshots = []

        for line in lines:
            snapshots.append(json.loads(line))


        agent2location = snapshots[-1]['agent2location']
        location2agent = snapshots[-1]['location2agent']
        weights = snapshots[-1]['weights']
        agents = snapshots[-1]['agents']

        num_iters = num_iters - len(snapshots)

    else:
        f = open(filename, 'w+')
        snapshots = []

        # Holds the location of each agent
        agent2location = {}
        location2agent = collections.defaultdict(set)

        location_names = list(locations.keys())

        # Initialize agents randomly at various locations
        for agent in agents:
            agent2location[agent] = random.choice(location_names)
            location2agent[agent2location[agent]].add(agent)

        weights = collections.defaultdict(int)

    for t in range(num_iters): 
        print(f'==== Round {t + 1} of {num_iters} ====')

        for agent in agents:
            
            # Decide on action (change location, or talk)
            if len(location2agent[agent2location[agent]]) <= 1 or np.random.uniform() <= p_change:
                # Change location
                location2agent[agent2location[agent]].remove(agent)
                agent2location[agent] = random.choice(location_names)
                location2agent[agent2location[agent]].add(agent)
                print(f'Round {t +1} : {agent} changes location')

            elif agents[agent]['is_student']:
                # Talk to random agent that is co-located with you
                neighbor = random.choice(list(location2agent[agent2location[agent]] - set([agent])))
                print(f'Round {t +1} : {agent} is talking to {neighbor}')
                agents[agent] = update_opinion(agents[agent], agents[neighbor])
                weights[agent, neighbor] += (100 - abs(agents[agent]['value'] - agents[neighbor]['value']))

        snapshot = {
            "agents" : copy.deepcopy(agents),
            "agent2location" : copy.deepcopy(agent2location),
            "location2agent" : copy.deepcopy(location2agent),
            "weights" : copy.deepcopy(weights)
        }

        snapshots.append(snapshot)

        f.write(json.dumps(snapshot) + '\n')
        f.flush()

    f.close()
        
    return snapshots

def plot_snapshots(snapshots, num_agents, num_iters):

    values = np.zeros((num_iters, num_agents))

    for t in range(num_iters):
        for i, agent in enumerate(snapshots[t]['agents'].keys()):
            values[t, i] = snapshots[t]['agents'][agent]['value']

    fig, ax = plt.subplots(figsize=(10, 10))

    for i in range(values.shape[-1]):
        ax.plot(1 + np.arange(num_iters), values[:, i])

    ax.set_xlabel('Round')
    ax.set_ylabel('Opinion')
    ax.set_ylim(0, 1)

    plt.savefig('opinions.pdf')

    graphs = []

    for t in range(num_iters):
        G = nx.Graph()
        G.add_nodes_from(snapshots[t]['agents'].keys())

        for agent1, agent2 in snapshots[t]['weights'].keys():
            G.add_edge(agent1, agent2, weight=snapshots[t]['weights'][agent1, agent2])

        plot_network(G)


def update_opinion(agent, neighbor):
    update_prompt = f"""
    # Task
    Your task is to update your opinion based on your existing opinion and the opinion of {neighbor['name']}.
    You must combine your public opinion, your private opinion, and the opinion of {neighbor['name']} to form a new public opinion.

    # Opinions
    Your opinions are given below after chevrons:
    <YOUR OPINION>
    {json.dumps(agent)}
    </YOUR OPINION>

    The public opinion of your neighbor is given below after chevrons:
    <NEIGHBOR OPINION>
    {json.dumps({'name' : neighbor['name'], 'public_opinion' : neighbor['public_opinion']})}
    </NEIGHBOR OPINION>

    # Output
    You should output a JSON object with the following fields:

    {{
        "public_opinion": the agent's public opiion after talking to the neighbor and a persuasive rationale for the opinion.
        "value" : (float) a numeric value ranging from 0 to 100, indicating the agent's new public opinion sentiment regarding the lesson fees (0 = support low fees, 100 = support high fees)
    }}

    ```json
    """

    response = json.loads(get_response(update_prompt).split('```')[0])

    agent['public_opinion'] = response['public_opinion']
    agent['value'] = response['value']

    return agent

In [15]:
num_agents = 32
num_classes = 3
num_iters = 2

config = create_world(num_classes=num_classes, num_agents=num_agents)

snapshots = network_formation(config['locations'], config['agents'], num_iters=num_agents)


plot_snapshots(snapshots, num_agents=num_agents, num_iters=num_iters)


Round 1 : Student 1 is talking to John Doe
Round 1 : Student 2 is talking to Student 8
Round 1 : Student 3 is talking to Student 20
Round 1 : Student 4 changes location
Round 1 : Student 5 is talking to Student 4
Round 1 : Student 6 is talking to Student 17
Round 1 : Student 7 is talking to Student 1
Round 1 : Student 8 is talking to Student 24
Round 1 : Student 9 is talking to Student 22
Round 1 : Student 10 is talking to Student 18
Round 1 : Student 11 is talking to Student 30
Round 1 : Student 12 changes location
Round 1 : Student 13 is talking to Student 31
Round 1 : Student 14 is talking to Student 5
Round 1 : Student 15 is talking to Student 23
